# Quickstart

This is a self contained guide on how to write a simple app and start launching
distributed jobs on local and remote clusters.

## Installation

First thing we need to do is to install the TorchX python package which includes
the CLI and the library.

```sh
# install torchx with all dependencies
$ pip install "torchx[dev]"
```

See the [README](https://github.com/meta-pytorch/torchx) for more
information on installation.

In [1]:
%%sh
torchx --help

usage: torchx [-h] [--log_level LOG_LEVEL] [--version]
              {builtins,cancel,configure,dele

te,describe,list,log,run,runopts,status,tracker}
              ...

torchx CLI

options:
  -h, --hel

p            show this help message and exit
  --log_level LOG_LEVEL
                        Python 

logging log level
  --version             show program's version number and exit

sub-commands:
  Us

e the following commands to run operations, e.g.: torchx run ${JOB_NAME}

  {builtins,cancel,configu

re,delete,describe,list,log,run,runopts,status,tracker}


## Hello World

Lets start off with writing a simple "Hello World" python app. This is just a
normal python program and can contain anything you'd like.

<div class="admonition note">
<div class="admonition-title">Note</div>
This example uses Jupyter Notebook `%%writefile` to create local files for
example purposes. Under normal usage you would have these as standalone files.
</div>

In [2]:
%%writefile my_app.py

import sys

print(f"Hello, {sys.argv[1]}!")

Overwriting my_app.py


## Launching

We can execute our app via `torchx run`. The
`local_cwd` scheduler executes the app relative to the current directory.

For this we'll use the `utils.python` component:

In [3]:
%%sh
torchx run --scheduler local_cwd utils.python --help

usage: torchx run <run args...> python  [--help] [-m str] [-c str]
                                 

       [--script str] [--image str]
                                        [--name str] [--cpu int]

 [--gpu int]
                                        [--memMB int] [-h str]
                        

                [--num_replicas int]
                                        ...

Runs ``python`` wi

th the specified module, command or script on the specified
image and host. Use ``--`` to separate c

...

 the job (default: torchx_utils_python)
  --cpu int           number of cpus per replica (default: 1

)
  --gpu int           number of gpus per replica (default: 0)
  --memMB int         cpu memory in 

MB per replica (default: 1024)
  -h str, --h str     a registered named resource (if specified takes


                      precedence over cpu, gpu, memMB) (default: None)
  --num_replicas int  number

 of copies to run (each on its own container)
                      (default: 1)


The component takes in the script name and any extra arguments will be passed to
the script itself.

In [4]:
%%sh
torchx run --scheduler local_cwd utils.python --script my_app.py "your name"

torchx 2026-02-02 23:09:54 INFO     Tracker configurations: {}


torchx 2026-02-02 23:09:54 INFO     Log directory not set in scheduler cfg. Creating a temporary log

 dir that will be deleted on exit. To preserve log directory set the `log_dir` cfg option
torchx 202

6-02-02 23:09:54 INFO     Log directory is: /tmp/torchx_6i7c6etf


torchx 2026-02-02 23:09:54 INFO     Waiting for the app to finish...


python/0 Hello, your name!


torchx 2026-02-02 23:09:55 INFO     Job finished: SUCCEEDED


local_cwd://torchx/torchx_utils_python-qvp5v7mq2nqlvd


We can run the exact same app via the `local_docker` scheduler. This scheduler
will package up the local workspace as a layer on top of the specified image.
This provides a very similar environment to the container based remote
schedulers.

<div class="admonition note">
<div class="admonition-title">Note</div>
This requires Docker installed and won't work in environments such as Google
Colab. See the Docker install instructions:
[https://docs.docker.com/get-docker/](https://docs.docker.com/get-docker/)</a>
</div>

In [5]:
%%sh
torchx run --scheduler local_docker utils.python --script my_app.py "your name"

torchx 2026-02-02 23:09:56 INFO     Tracker configurations: {}


torchx 2026-02-02 23:09:56 INFO     Workspace `/home/runner/work/torchx/torchx/docs/source` resolved

 to filesystem path `/home/runner/work/torchx/torchx/docs/source`


torchx 2026-02-02 23:09:56 INFO     Building workspace docker image (this may take a while)...


torchx 2026-02-02 23:09:56 INFO     Step 1/4 : ARG IMAGE


...

es


torchx 2026-02-02 23:10:09 INFO     Waiting for the app to finish...


python/0 Hello, your name!


torchx 2026-02-02 23:10:10 INFO     Job finished: SUCCEEDED


local_docker://torchx/torchx_utils_python-qgbvnmkp6xd7pd


TorchX defaults to using the
[ghcr.io/pytorch/torchx](https://ghcr.io/pytorch/torchx) Docker container image
which contains the PyTorch libraries, TorchX and related dependencies.

## Distributed

TorchX's `dist.ddp` component uses
[TorchElastic](https://pytorch.org/docs/stable/distributed.elastic.html)
to manage the workers. This means you can launch multi-worker and multi-host
jobs out of the box on all of the schedulers we support.

In [6]:
%%sh
torchx run --scheduler local_docker dist.ddp --help

usage: torchx run <run args...> ddp  [--help] [--script str] [-m str]
                              

       [--image str] [--name str] [-h str]
                                     [--cpu int] [--gpu i

nt] [--memMB int]
                                     [-j str] [--env str] [--metadata str]
       

                              [--max_retries int] [--rdzv_port int]
                                

     [--rdzv_backend str] [--rdzv_conf str]
                                     [--mounts str] [--d

...

                     (ex. type=<bind/volume>,src=/host,dst=/job[,readonly]).
                      S

ee scheduler documentation for more info. (default:
                      None)
  --debug str       

  whether to run with preset debug flags enabled (default:
                      False)
  --tee int 

          tees the specified std stream(s) to console + file. 0:
                      none, 1: stdo

ut, 2: stderr, 3: both (default: 3)


Lets create a slightly more interesting app to leverage the TorchX distributed
support.

In [7]:
%%writefile dist_app.py

import torch
import torch.distributed as dist

dist.init_process_group(backend="gloo")
print(f"I am worker {dist.get_rank()} of {dist.get_world_size()}!")

a = torch.tensor([dist.get_rank()])
dist.all_reduce(a)
print(f"all_reduce output = {a}")

Writing dist_app.py


Let launch a small job with 2 nodes and 2 worker processes per node:

In [8]:
%%sh
torchx run --scheduler local_docker dist.ddp -j 2x2 --script dist_app.py

torchx 2026-02-02 23:10:12 INFO     Tracker configurations: {}


torchx 2026-02-02 23:10:12 INFO     Workspace `/home/runner/work/torchx/torchx/docs/source` resolved

 to filesystem path `/home/runner/work/torchx/torchx/docs/source`


torchx 2026-02-02 23:10:13 INFO     Building workspace docker image (this may take a while)...


torchx 2026-02-02 23:10:13 INFO     Step 1/4 : ARG IMAGE


...

pp/0 [0]:I am worker 0 of 4!
dist_app/0 [0]:all_reduce output = tensor([6])
dist_app/0 [1]:[Gloo] Ra

nk 1 is connected to 3 peer ranks. Expected number of connected peer ranks is : 3
dist_app/0 [1]:I a

m worker 1 of 4!
dist_app/0 [1]:all_reduce output = tensor([6])


torchx 2026-02-02 23:10:32 INFO     Job finished: SUCCEEDED


local_docker://torchx/dist_app-cpm9mr4vrphz2c


## Workspaces / Patching

For each scheduler there's a concept of an `image`. For `local_cwd` and `slurm`
it uses the current working directory. For container based schedulers such as
`local_docker`, `kubernetes` and `aws_batch` it uses a docker container.

To provide the same environment between local and remote jobs, TorchX CLI uses
workspaces to automatically patch images for remote jobs on a per scheduler
basis.

When you launch a job via `torchx run` it'll overlay the current directory on
top of the provided image so your code is available in the launched job.

For `docker` based schedulers you'll need a local docker daemon to build and
push the image to your remote docker repository.

## `.torchxconfig`

Arguments to schedulers can be specified either via a command line flag to
`torchx run -s <scheduler> -c <args>` or on a per scheduler basis via a
`.torchxconfig` file.

In [9]:
%%writefile .torchxconfig

[kubernetes]
queue=torchx
image_repo=<your docker image repository>

[slurm]
partition=torchx

Writing .torchxconfig


## Remote Schedulers

TorchX supports a large number of schedulers.
Don't see yours?
[Request it!](https://github.com/meta-pytorch/torchx/issues/new?assignees=&labels=&template=feature-request.md)

Remote schedulers operate the exact same way the local schedulers do. The same
run command for local works out of the box on remote.

```sh
$ torchx run --scheduler slurm dist.ddp -j 2x2 --script dist_app.py
$ torchx run --scheduler kubernetes dist.ddp -j 2x2 --script dist_app.py
$ torchx run --scheduler aws_batch dist.ddp -j 2x2 --script dist_app.py
```

Depending on the scheduler there may be a few extra configuration parameters so
TorchX knows where to run the job and upload built images. These can either be
set via `-c` or in the `.torchxconfig` file.

All config options:

In [10]:
%%sh
torchx runopts

local_docker:
    usage:
        [copy_env=COPY_ENV],[env=ENV],[privileged=PRIVILEGED],[image_repo=I

MAGE_REPO],[quiet=QUIET]

    optional arguments:
        copy_env=COPY_ENV (list, None)
           

 list of glob patterns of environment variables to copy if not set in AppDef. Ex: FOO_*
        env=

ENV (dict, None)
            environment variables to be passed to the run. The separator sign can b

e eiher comma or semicolon
            (e.g. ENV1:v1,ENV2:v2,ENV3:v3 or ENV1:V1;ENV2:V2). Environmen

...

E (str, None)
            queue name to submit jobs
        jobdir=JOBDIR (str, None)
            Th

e directory to place the job code and outputs. The directory must not exist and will be created.
   

     container_workdir=CONTAINER_WORKDIR (str, None)
            working directory in container jobs


        host_network=HOST_NETWORK (bool, False)
            True if using the host network for jobs


        shm_size=SHM_SIZE (str, 64m)
            size of shared memory (/dev/shm) for jobs



## Custom Images

### Docker-based Schedulers

If you want more than the standard PyTorch libraries you can add custom
Dockerfile or build your own docker container and use it as the base image for
your TorchX jobs.


In [11]:
%%writefile timm_app.py

import timm

print(timm.models.resnet18())

Writing timm_app.py


In [12]:
%%writefile Dockerfile.torchx

FROM pytorch/pytorch:1.10.0-cuda11.3-cudnn8-runtime

RUN pip install timm

COPY . .

Writing Dockerfile.torchx


Once we have the Dockerfile created we can launch as normal and TorchX will
automatically build the image with the newly provided Dockerfile instead of the
default one.

In [13]:
%%sh
torchx run --scheduler local_docker utils.python --script timm_app.py

torchx 2026-02-02 23:10:35 INFO     loaded configs from /home/runner/work/torchx/torchx/docs/source/

.torchxconfig


torchx 2026-02-02 23:10:35 INFO     Tracker configurations: {}


torchx 2026-02-02 23:10:35 INFO     Workspace `/home/runner/work/torchx/torchx/docs/source` resolved

 to filesystem path `/home/runner/work/torchx/torchx/docs/source`


...

  )
python/0   )
python/0   (global_pool): SelectAdaptivePool2d(pool_type=avg, flatten=Flatten(start

_dim=1, end_dim=-1))
python/0   (fc): Linear(in_features=512, out_features=1000, bias=True)
python/0

 )


torchx 2026-02-02 23:11:58 INFO     Job finished: SUCCEEDED


local_docker://torchx/torchx_utils_python-h4hc2wlpdv2rbd


### Slurm

The `slurm` and `local_cwd` use the current environment so you can use `pip` and
`conda` as normal.

## Next Steps

1. Checkout other features of the [torchx CLI](cli.rst)
2. Take a look at the [list of schedulers](schedulers.rst) supported by the runner
3. Browse through the collection of [builtin components](components/overview.rst)
5. See a [training app example](examples_apps/index.rst)